In [4]:

import math, json, re, ccxt
from pprint import pformat
wex = ccxt.wex()
data_for_graph = {}
data = wex.fetch_tickers()
for key in data:
    data_for_graph.update({key: data[key]['last']})
ticker_list = list(data_for_graph.keys())
ticker_list2 = []
for key in ticker_list:
    ticker_list2.append("{1}/{0}".format(*key.split('/'))) #обратные тикеры

for key in range(0, len(ticker_list)):
    #обратные котировки
    data_for_graph.update({ticker_list2[key]: 1 / (data[ticker_list[key]]['last'])})

print(pformat(data_for_graph))


{'BCH/BCHET': 1.5015015015015014,
 'BCH/BTC': 0.1131,
 'BCH/DASH': 2.378,
 'BCH/ETH': 1.888,
 'BCH/EUR': 756.018,
 'BCH/LTC': 6.159,
 'BCH/RUR': 52325.205,
 'BCH/USD': 927.471,
 'BCH/ZEC': 4.087,
 'BCHET/BCH': 0.666,
 'BTC/BCH': 8.841732979664014,
 'BTC/BTCET': 1.5151515151515151,
 'BTC/DASH': 21.04820037886761,
 'BTC/ETH': 16.70285618840822,
 'BTC/EUR': 6697.16213,
 'BTC/LTC': 54.614964500273075,
 'BTC/NMC': 5000.0,
 'BTC/NVC': 2127.6595744680853,
 'BTC/PPC': 4347.826086956522,
 'BTC/RUR': 462427.13214,
 'BTC/USD': 8207.505,
 'BTC/ZEC': 36.101083032490976,
 'BTCET/BTC': 0.66,
 'DASH/BCH': 0.42052144659377627,
 'DASH/BTC': 0.04751,
 'DASH/DSHET': 1.402524544179523,
 'DASH/ETH': 0.792,
 'DASH/EUR': 317.192,
 'DASH/LTC': 2.599,
 'DASH/RUR': 22023.389,
 'DASH/USD': 389.02454,
 'DASH/ZEC': 1.716,
 'DSHET/DASH': 0.713,
 'ETH/BCH': 0.5296610169491526,
 'ETH/BTC': 0.05987,
 'ETH/DASH': 1.2626262626262625,
 'ETH/ETHET': 1.6611295681063123,
 'ETH/EUR': 402.28562,
 'ETH/LTC': 3.26881,
 'ETH/RUR'

Создаем граф в виде списка смежности для каждой ноды - тикера криптовалюты, 
котировки переводим в логарифмический вид.

In [7]:
graph = {}
jsrates = data_for_graph

pattern = re.compile("([A-Z]{3,5})/([A-Z]{3,5})")

try:
    for key in jsrates:
        matches = pattern.match(key)

        conversion_rate = math.log(float(jsrates[key]))
        
        from_rate = matches.group(1).encode('ascii', 'ignore')

        to_rate = matches.group(2).encode('ascii', 'ignore')

        if from_rate != to_rate:
            if from_rate not in graph:
                graph[from_rate] = {}
            graph[from_rate][to_rate] = float(conversion_rate)

except AttributeError:
    print('Strange rate')
    pass

print(pformat(graph))
# воплощение алгоритма отсюда
#  https://gist.github.com/joninvski/701720


In [8]:


# Step 1: For each node prepare the destination and predecessor
def initialize(graph, source):
    d = {}  # Stands for destination
    p = {}  # Stands for predecessor
    for node in graph:
        d[node] = float('Inf')  # We start admiting that the rest of nodes are very very far
        p[node] = None
    d[source] = 0  # For the source we know how to reach
    return d, p


def relax(node, neighbour, graph, d, p):
    # If the distance between the node and the neighbour is lower than the one I have now
    if d[neighbour] > d[node] + graph[node][neighbour]:
        # Record this lower distance
        d[neighbour] = d[node] + graph[node][neighbour]
        p[neighbour] = node


def retrace_negative_loop(p, start):
    arbitrageLoop = [start]
    next_node = start
    while True:
        next_node = p[next_node]
        if next_node not in arbitrageLoop:
            arbitrageLoop.append(next_node)
        else:
            arbitrageLoop.append(next_node)
            arbitrageLoop = arbitrageLoop[arbitrageLoop.index(next_node):]
            return arbitrageLoop


def bellman_ford(graph, source):
    d, p = initialize(graph, source)
    for i in range(len(graph) - 1):  # Run this until is converges
        for u in graph:
            for v in graph[u]:  # For each neighbour of u
                relax(u, v, graph, d, p)  # Lets relax it

    # Step 3: check for negative-weight cycles
    for u in graph:
        for v in graph[u]:
            if d[v] < d[u] + graph[u][v]:
                return (retrace_negative_loop(p, source))
    return None


In [9]:
 paths = []
 for key in graph:

    path = bellman_ford(graph, key)

    if path not in paths and not None and len(path)>3:
        paths.append(path)

for path in paths:
    if path == None:
        print("No opportunity here :(")
    else:
        
        money = 100
        print("Starting with %(money)i in %(currency)s" % {"money": money, "currency": path[0]})           

        for i, value in enumerate(path):
            if i + 1 < len(path):
                start = path[i]
                end = path[i + 1]
                rate = math.exp(graph[start][end])
                money *= rate
                print("%(start)s to %(end)s at %(rate)f = %(money)f" % {"start": start, "end": end, "rate": rate,
                                                                        "money": money})
print ("\n")

 

Starting with 100 in b'BTC'
b'BTC' to b'DASH' at 21.048200 = 2104.820038
b'DASH' to b'BCH' at 0.420521 = 885.121967
b'BCH' to b'BTC' at 0.113100 = 100.107294
Starting with 100 in b'NVC'
b'NVC' to b'USD' at 3.900000 = 390.000000
b'USD' to b'DASH' at 0.002571 = 1.002507
b'DASH' to b'BCH' at 0.420521 = 0.421576
b'BCH' to b'BTC' at 0.113100 = 0.047680
b'BTC' to b'NVC' at 2127.659574 = 101.447303
Starting with 100 in b'USD'
b'USD' to b'DASH' at 0.002571 = 0.257053
b'DASH' to b'BCH' at 0.420521 = 0.108096
b'BCH' to b'BTC' at 0.113100 = 0.012226
b'BTC' to b'NVC' at 2127.659574 = 26.012129
b'NVC' to b'USD' at 3.900000 = 101.447303
Starting with 100 in b'BCH'
b'BCH' to b'BTC' at 0.113100 = 11.310000
b'BTC' to b'NVC' at 2127.659574 = 24063.829787
b'NVC' to b'USD' at 3.900000 = 93848.936170
b'USD' to b'DASH' at 0.002571 = 241.241687
b'DASH' to b'BCH' at 0.420521 = 101.447303
Starting with 100 in b'BTC'
b'BTC' to b'NVC' at 2127.659574 = 212765.957447
b'NVC' to b'USD' at 3.900000 = 829787.234043
b'